In [1]:
import sys
if 'google.colab' in sys.modules:
    !git clone  https://github.com/ecastillot/delaware.git ./delaware
    !pip install obspy
    

In [2]:
import sys
import os

version = "10102024"

if 'google.colab' in sys.modules:
    dw_path = os.path.join("/content/delaware",version)
else:
    dw_path = os.path.join("/home/emmanuel/ecastillo/dev/delaware",version)
    
sys.path.append(dw_path)

# Stations

In [3]:
import pandas as pd
import os

stations_relpath = "data_git/stations/standard_stations.csv"
stations_path = os.path.join(dw_path,stations_relpath)
stations = pd.read_csv(stations_path)
stations_columns = ["network","station","latitude","longitude","elevation","x[km]","y[km]"]
stations = stations[stations_columns]
stations

,network,station,latitude,longitude,elevation,x[km],y[km]
0,4O,WB01,31.721667,-104.060278,0.92700,-11583.937159,3726.830293
1,4O,CT01,31.902850,-104.144532,0.99400,-11593.316271,3750.564925
2,4O,SA02,31.671630,-104.264900,0.76000,-11606.715576,3720.283725
3,4O,SA04,31.755930,-104.254580,0.76000,-11605.566758,3731.315115
4,4O,SA06,31.756960,-104.149710,0.76000,-11593.892683,3731.449961
5,4O,WB02,31.736302,-103.944920,0.93200,-11571.095621,3728.745711
6,4O,WB03,31.610722,-103.968839,0.99400,-11573.758171,3712.319547
7,4O,SA07,31.865780,-104.364810,1.10300,-11617.837506,3745.705033
8,4O,WB04,31.530513,-103.950319,0.96189,-11571.696579,3701.839715
9,4O,WB05,31.753781,-103.832948,0.85481,-11558.630888,3731.033735


# Custom Client: Catalog with picks

In [12]:
from obspy import UTCDateTime
from delaware.core.client import CustomClient
from delaware.loc.inv import prepare_cat2inv

provider = "USGS"
sta_ref = "WB03"
radius = 1
starttime = UTCDateTime(f"2024-01-01 15:26:30")
endtime = UTCDateTime("2024-05-30 15:26:32")

client =  CustomClient(provider)

ref = stations[stations["station"].isin([sta_ref])]
ref = ref.drop_duplicates(subset=["station"],
                                          ignore_index=True)

cat,picks,mag = client.get_custom_events(
                        latitude=ref.loc[0].latitude,
                        longitude=ref.loc[0].longitude,
                        maxradius=radius/111,
                        includeallorigins=True,
                        starttime=starttime,
                        endtime=endtime)

cat, picks = prepare_cat2inv(cat,picks,attach_station=stations)
print("### catalog ###\n",cat)
print("### picks ###\n",picks)

### catalog ###
              ev_id                origin_time  eq_latitude  eq_longitude  \
0   texnet2024abrd 2024-01-01 21:47:07.938951       31.609      -103.976   
1   texnet2024absh 2024-01-01 22:22:45.982513       31.615      -103.971   
2   texnet2024agra 2024-01-04 15:25:30.684960       31.614      -103.961   
3   texnet2024anyy 2024-01-08 15:22:08.156378       31.609      -103.963   
4   texnet2024augx 2024-01-12 02:12:02.773522       31.605      -103.966   
5   texnet2024bwbc 2024-01-27 07:04:20.072420       31.612      -103.966   
6   texnet2024cegj 2024-01-31 18:47:54.441996       31.612      -103.976   
7   texnet2024ctha 2024-02-09 00:12:40.809245       31.610      -103.968   
8   texnet2024cthh 2024-02-09 00:20:59.750865       31.609      -103.969   
9   texnet2024cxjd 2024-02-11 05:48:25.033211       31.617      -103.968   
10  texnet2024cyjh 2024-02-11 19:01:34.699864       31.617      -103.968   
11  texnet2024czag 2024-02-12 03:35:45.632085       31.616      -103.96

/home/emmanuel/ecastillo/dev/delaware/10102024/delaware/core/client.py:712: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_picks = pd.concat(all_picks, axis=0)


In [13]:
picks = picks[picks["station"]==sta_ref]
picks

,ev_id,station,r,az,tt_P,tt_S
0,texnet2024cegj,WB03,0.694171,101.781674,0.846753,3.480092
3,texnet2024agra,WB03,0.827874,243.955523,1.210039,2.527744
4,texnet2024hwjm,WB03,0.890044,77.613680,1.229519,2.474192
11,texnet2024ejzn,WB03,0.962305,64.605943,1.608898,2.890856
12,texnet2024absh,WB03,0.516835,156.619821,1.622953,2.768295
13,texnet2024cxjd,WB03,0.700693,186.520918,1.625674,2.768279
14,texnet2024gdrz,WB03,0.831161,68.709728,1.630815,2.857776
15,texnet2024cyjh,WB03,0.700693,186.520918,1.645802,2.791631
16,texnet2024czey,WB03,0.785688,207.619704,1.645865,2.841477
17,texnet2024ekap,WB03,0.680247,92.599983,1.657705,3.02947


# Particle Swarm Optimization

In [16]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess

    # Compute the errors (residuals)
    tp_error = (tp_pred - tp_obs)**2
    ts_error = (ts_pred - ts_obs)**2
    loss = tp_error + ts_error

    return loss

def vd_pso(cost_func,
           picks,
           bounds,
           max_iter=100,w=0.5,c1=0.8,c2=0.9):
    
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    velocities = np.zeros((n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r",'tt_P','tt_S']}
    
    # Initialize the particles and velocities
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, 3))
    velocities = np.zeros((n_particles, 3))

    # Initialize the best positions and best costs
    best_positions = particles.copy()
    best_costs = cost_func(
                            particles=particles,
                            r=args["r"],
                            tp_obs=args["tt_P"],
                            ts_obs=args["tt_S"]
                        )

    # Initialize the global best position and global best cost
    global_best_position = particles[0].copy()
    global_best_cost = best_costs[0]

    # Perform the optimization
    for i in range(max_iter):
        # Update the velocities
        r1 = np.random.rand(n_particles, 3) #Random matrix used to compute the cognitive component of the veocity update
        r2 = np.random.rand(n_particles, 3) #Random matrix used to compute the social component of the veocity update


        #Cognitive component is calculated by taking the difference between the
        #particle's current position and its best personal position found so far,
        #and then multiplying it by a random matrix r1 and a cognitive acceleration coefficient c1.
        cognitive_component = c1 * r1 * (best_positions - particles)

        #The social component represents the particle's tendency to move towards the
        #global best position found by the swarm. It is calculated by taking the
        #difference between the particle's current position and the global best position
        # found by the swarm, and then multiplying it by a random matrix r2 and a
        #social acceleration coefficient c2.
        social_component = c2 * r2 * (global_best_position - particles)

        #The new velocity of the particle is computed by adding the current velocity
        #to the sum of the cognitive and social components, multiplied by the inertia
        #weight w. The new velocity is then used to update the position of the
        #particle in the search space.
        velocities = w * velocities + cognitive_component + social_component

        # Update the particles
        particles += velocities

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate the objective function
        costs = cost_func(
                            particles=particles,
                            r=args["r"],
                            tp_obs=args["tt_P"],
                            ts_obs=args["tt_S"]
                        )

        # Update the best positions and best costs
        is_best = costs < best_costs
        best_positions[is_best] = particles[is_best]
        best_costs[is_best] = costs[is_best]

        # Update the global best position and global best cost
        global_best_index = np.argmin(best_costs)
        global_best_position = best_positions[global_best_index].copy()
        global_best_cost = best_costs[global_best_index]

        # Print the progress
        if (i+1)%10 == 0:
            print(f'Iteration {i+1}: Best Cost = {global_best_cost} | sol:{global_best_position}')
    
    return global_best_position, global_best_cost



cost_func = loss_function


bounds = np.array([[0,2,2/2],[10,4.7,4.7/2]])

# Run the PSO algorithm on the Rastrigin function
solution, fitness = vd_pso(cost_func=cost_func,picks=picks,
                            bounds=bounds,max_iter=500
                            )

print("Best Solution [z,vp,vs]:", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 0.00010206645509928494 | sol:[5.0893537  2.56241267 1.48201407]
Iteration 20: Best Cost = 1.1771798909209186e-05 | sol:[5.05903949 2.53752283 1.48455118]
Iteration 30: Best Cost = 7.316952394359486e-06 | sol:[5.05712693 2.53809454 1.48344438]
Iteration 40: Best Cost = 7.314086975579144e-06 | sol:[5.05712472 2.53809504 1.48344302]
Iteration 50: Best Cost = 1.4629271088542764e-06 | sol:[5.04130771 2.52601059 1.48390465]
Iteration 60: Best Cost = 1.4535939511678488e-06 | sol:[5.04129754 2.52601182 1.48391175]
Iteration 70: Best Cost = 1.4511905649544903e-06 | sol:[5.04129549 2.52601234 1.48391276]
Iteration 80: Best Cost = 1.4511882326755408e-06 | sol:[5.04129549 2.52601234 1.48391276]
Iteration 90: Best Cost = 1.4511882303990667e-06 | sol:[5.04129549 2.52601234 1.48391276]
Iteration 100: Best Cost = 1.4511882303972543e-06 | sol:[5.04129549 2.52601234 1.48391276]
Iteration 110: Best Cost = 1.4511882303972543e-06 | sol:[5.04129549 2.52601234 1.48391276]
Iteration 

# Gradient Descent

In [21]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess
    loss = ((tp_pred - tp_obs)**2) + ((ts_pred - ts_obs)**2)
    return loss

def gradient(loss_func, particles, r, tp_obs, ts_obs, epsilon=1e-6):
    """Calculate gradients for the loss function w.r.t. particles."""
    gradients = np.zeros_like(particles)
    for i in range(particles.shape[1]):
        # Perturb each parameter by a small epsilon and calculate the gradient
        particles_plus = particles.copy()
        particles_plus[:, i] += epsilon
        loss_plus = loss_func(particles_plus, r, tp_obs, ts_obs)
        
        particles_minus = particles.copy()
        particles_minus[:, i] -= epsilon
        loss_minus = loss_func(particles_minus, r, tp_obs, ts_obs)
        
        # Compute the gradient (difference between perturbed losses)
        gradients[:, i] = (loss_plus - loss_minus) / (2 * epsilon)
        
    return gradients

def vd_gradient_descent(cost_func, picks, bounds, max_iter=100, lr=0.01):
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r", 'tt_P', 'tt_S']}
    
    # Initialize the best position and best cost
    costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])
    best_cost = np.min(costs)
    best_position = particles[np.argmin(costs)].copy()

    # Perform gradient descent optimization
    for i in range(max_iter):
        # Compute gradients of the loss function
        gradients = gradient(cost_func, particles, args["r"], args["tt_P"], args["tt_S"])
        
        # Update particles using gradient descent
        particles -= lr * gradients

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate the new costs
        costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])

        # Update best position if new cost is lower
        current_best_cost = np.min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_position = particles[np.argmin(costs)].copy()

        # Print progress
        if (i + 1) % 10 == 0:
            print(f'Iteration {i+1}: Best Cost = {best_cost} | sol:{best_position}')

    return best_position, best_cost

# Example of how to call the Gradient Descent-based optimization function
bounds = np.array([[0,2,2/2],[10,4.7,4.7/2]])  # bounds for z_guess, vp_guess, vs_guess

# Call the function with your data (picks_ready should be a DataFrame or similar)
solution, fitness = vd_gradient_descent(cost_func=loss_function, picks=picks, bounds=bounds, max_iter=500, lr=0.01)

print("Best Solution [z,vp,vs]:", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 0.08380382324357293 | sol:[4.43140102 2.17046161 1.36213979]
Iteration 20: Best Cost = 0.02527675032153201 | sol:[6.68442816 3.32564655 1.88159649]
Iteration 30: Best Cost = 0.007020354153517396 | sol:[6.67002467 3.33210824 1.92102195]
Iteration 40: Best Cost = 0.0029186526779155344 | sol:[6.66211854 3.33770158 1.93873327]
Iteration 50: Best Cost = 0.0017562569776356596 | sol:[6.65724405 3.34266493 1.94694694]
Iteration 60: Best Cost = 0.001305166001824124 | sol:[6.6539101  3.34712903 1.95069593]
Iteration 70: Best Cost = 0.0010543462032406574 | sol:[6.65141241 3.35117473 1.9522858 ]
Iteration 80: Best Cost = 0.0008759328394162852 | sol:[6.64940149 3.35485753 1.95282556]
Iteration 90: Best Cost = 0.0007344126583934045 | sol:[6.64769784 3.35821893 1.95286069]
Iteration 100: Best Cost = 0.0006177491795943275 | sol:[6.64620611 3.36129214 1.95266177]
Iteration 110: Best Cost = 0.000520334998193125 | sol:[6.64487343 3.36410502 1.95236297]
Iteration 120: Best Cost =

# Adam

In [25]:
import numpy as np

def loss_function(particles, r, tp_obs, ts_obs):
    z_guess = particles[:, 0]
    vp_guess = particles[:, 1]
    vs_guess = particles[:, 2]

    if np.any(vp_guess == 0):
        raise Exception("vp_guess must not contain any zeros")
    if np.any(vs_guess == 0):
        raise Exception("vs_guess must not contain any zeros")

    tp_pred = np.sqrt(r**2 + z_guess**2) / vp_guess
    ts_pred = np.sqrt(r**2 + z_guess**2) / vs_guess
    loss = ((tp_pred - tp_obs)**2) + ((ts_pred - ts_obs)**2)
    return loss

def vd_adam(cost_func, picks, bounds, max_iter=100, lr=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
    n_particles = len(picks)
    dim = bounds.shape[-1]
    
    particles = np.random.uniform(low=bounds[0], high=bounds[1], size=(n_particles, dim))
    args = {col: picks[col].to_numpy() for col in ["r", 'tt_P', 'tt_S']}
    
    # Initialize moment estimates
    m = np.zeros_like(particles)  # First moment (mean of gradients)
    v = np.zeros_like(particles)  # Second moment (uncentered variance of gradients)
    t = 0  # Time step (for bias correction)

    # Evaluate initial cost
    costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])
    best_cost = np.min(costs)
    best_position = particles[np.argmin(costs)].copy()

    # Perform Adam optimization
    for i in range(max_iter):
        t += 1  # Increment time step

        # Compute gradients (numerical approximation)
        gradients = gradient(cost_func, particles, args["r"], args["tt_P"], args["tt_S"])

        # Update moment estimates (bias correction applied to m and v)
        m = beta1 * m + (1 - beta1) * gradients  # Update first moment
        v = beta2 * v + (1 - beta2) * gradients**2  # Update second moment

        # Bias correction (to counteract initialization bias)
        m_hat = m / (1 - beta1**t)
        v_hat = v / (1 - beta2**t)

        # Update particles using Adam's update rule
        particles -= lr * m_hat / (np.sqrt(v_hat) + epsilon)

        # Enforce the bounds of the search space
        particles = np.clip(particles, bounds[0], bounds[1])

        # Evaluate new costs
        costs = cost_func(particles=particles, r=args["r"], tp_obs=args["tt_P"], ts_obs=args["tt_S"])

        # Update best position if new cost is lower
        current_best_cost = np.min(costs)
        if current_best_cost < best_cost:
            best_cost = current_best_cost
            best_position = particles[np.argmin(costs)].copy()

        # Print progress
        if (i + 1) % 10 == 0:
            print(f'Iteration {i+1}: Best Cost = {best_cost:.6f} | sol:{best_position}')

    return best_position, best_cost

def gradient(loss_func, particles, r, tp_obs, ts_obs, epsilon=1e-6):
    """Calculate gradients for the loss function w.r.t. particles."""
    gradients = np.zeros_like(particles)
    for i in range(particles.shape[1]):
        # Perturb each parameter by a small epsilon and calculate the gradient
        particles_plus = particles.copy()
        particles_plus[:, i] += epsilon
        loss_plus = loss_func(particles_plus, r, tp_obs, ts_obs)
        
        particles_minus = particles.copy()
        particles_minus[:, i] -= epsilon
        loss_minus = loss_func(particles_minus, r, tp_obs, ts_obs)
        
        # Compute the gradient (difference between perturbed losses)
        gradients[:, i] = (loss_plus - loss_minus) / (2 * epsilon)
        
    return gradients

# Example of how to call the Adam-based optimization function
bounds = np.array([[0,2,2/2],[10,4.7,4.7/2]])  # bounds for z_guess, vp_guess, vs_guess

# Call the function with your data (picks_ready should be a DataFrame or similar)
solution, fitness = vd_adam(cost_func=loss_function, picks=picks, bounds=bounds, max_iter=500, lr=0.001)

print("Best Solution: [z,vp,vs]", solution)
print("Best Fitness:", fitness)

Iteration 10: Best Cost = 0.034294 | sol:[4.39017728 2.67087213 1.37515904]
Iteration 20: Best Cost = 0.028787 | sol:[4.38437742 2.66089991 1.38444613]
Iteration 30: Best Cost = 0.024559 | sol:[4.38666278 2.65100163 1.39265629]
Iteration 40: Best Cost = 0.020936 | sol:[4.39524053 2.64126944 1.39970352]
Iteration 50: Best Cost = 0.017738 | sol:[4.40635558 2.63180303 1.40580777]
Iteration 60: Best Cost = 0.014923 | sol:[4.41807159 2.62267643 1.41124637]
Iteration 70: Best Cost = 0.012461 | sol:[4.42957175 2.61393601 1.41622715]
Iteration 80: Best Cost = 0.010322 | sol:[4.44053583 2.60560936 1.42086783]
Iteration 90: Best Cost = 0.008477 | sol:[4.45086286 2.59771331 1.42521864]
Iteration 100: Best Cost = 0.006900 | sol:[4.46054043 2.59025897 1.42929288]
Iteration 110: Best Cost = 0.005564 | sol:[4.46958212 2.58325427 1.43309038]
Iteration 120: Best Cost = 0.004443 | sol:[4.47800151 2.57670479 1.4366105 ]
Iteration 130: Best Cost = 0.003512 | sol:[4.48580617 2.57061364 1.43985673]
Iteratio